In [ ]:
import requests
from solana.rpc.api import Client

# Set up Solana RPC & Solscan API
SOLANA_RPC_URL = "https://api.mainnet-beta.solana.com"
SOLSCAN_API_BASE = "https://public-api.solscan.io"
client = Client(SOLANA_RPC_URL)

def get_token_metadata(token_address):
    """
    Fetch token metadata from Solscan API.
    """
    url = f"{SOLSCAN_API_BASE}/token/meta?tokenAddress={token_address}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

def check_token_ownership(token_address):
    """
    Check if the token's mint authority is still controlled by the developer.
    """
    try:
        response = client.get_account_info(token_address)
        if response and response['result']['value']:
            owner = response['result']['value']['owner']
            return owner
        return None
    except Exception as e:
        print(f"Error checking ownership: {e}")
        return None

def get_token_holders(token_address):
    """
    Fetch token holder distribution from Solscan API.
    """
    url = f"{SOLSCAN_API_BASE}/token/holders?tokenAddress={token_address}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get('data', [])
    return []

def detect_rug_pull_risks(token_address):
    """
    Analyze if the token has high rug pull risks.
    """
    metadata = get_token_metadata(token_address)
    if not metadata:
        return "Token not found on Solscan."

    mint_authority = check_token_ownership(token_address)
    holders = get_token_holders(token_address)

    risks = []

    # Check if contract is verified
    if not metadata.get("verified", False):
        risks.append("⚠️ Contract is NOT verified on Solscan.")

    # Check if mint authority still exists
    if mint_authority:
        risks.append(f"⚠️ Mint authority still exists: {mint_authority} (Developer might mint more tokens!)")

    # Check holder distribution
    if holders:
        top_holders = holders[:10]
        total_supply = sum(holder["amount"] for holder in holders)
        top_10_supply = sum(holder["amount"] for holder in top_holders)
        top_10_percentage = (top_10_supply / total_supply) * 100

        if top_10_percentage > 20:
            risks.append(f"⚠️ Top 10 wallets hold {top_10_percentage:.2f}% of the total supply.")

    # Check liquidity lock status (this might require a manual check)
    risks.append("🔍 Verify manually if LP tokens are burned or locked.")

    return risks if risks else ["✅ No major risks detected."]

# Example usage
token_address = "So11111111111111111111111111111111111111112"  # Replace with a real token address
risks_detected = detect_rug_pull_risks(token_address)

print("\n".join(risks_detected))


T
o
k
e
n
 
n
o
t
 
f
o
u
n
d
 
o
n
 
S
o
l
s
c
a
n
.
